In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/nanoGPT-song


/content/drive/MyDrive/nanoGPT-song


In [ ]:
ls

assets/          data/         newtrain.py   sample.py
bench.py         LICENSE       out-lyrics/   scaling_laws.ipynb
config/          losstrain.py  __pycache__/  train.py
configurator.py  model.py      README.md     transformer_sizing.ipynb


In [ ]:
!pip install torch numpy transformers datasets tiktoken wandb tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
!python data/lyrics/prepare.py


train has 22,308,928 tokens
val has 2,456,916 tokens


In [ ]:
!python losstrain.py config/train_lyrics.py \
  --device=cuda \
  --dtype=float32 \
  --compile=False \
  --max_iters=500 \
  --log_interval=1 \
  --eval_iters=20 \
  --n_layer=4 \
  --n_head=4 \
  --n_embd=256 \
  --batch_size=4 \
  --block_size=128 \
  --gradient_accumulation_steps=1


Overriding config with config/train_lyrics.py:
# train a miniature character-level model on song lyrics
# good for debugging and playing on macbooks and such

out_dir = 'out-lyrics'
eval_interval = 250  # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10  # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False  # override via command line if you like
wandb_project = 'lyrics'
wandb_run_name = 'mini-gpt'

dataset = 'lyrics'
batch_size = 64
block_size = 256  # context of up to 256 previous characters

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3  # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000  # make equal to max_iters usually
min_lr = 1e-4  # learning_rate / 10 usually
beta2 = 0.99  # make a bit bigger because number of tokens per iter is small

warmup_iters = 100  # not super nec

In [ ]:
!python sample.py \
  --device=cuda \
  --out_dir=out-lyrics \
  --init_from=resume \
  --start="LOVE IS" \
  --num_samples=3 \
  --max_new_tokens=100


Overriding: device = cuda
Overriding: out_dir = out-lyrics
Overriding: init_from = resume
Overriding: start = LOVE IS
Overriding: num_samples = 3
Overriding: max_new_tokens = 100
number of parameters: 16.03M
No meta.pkl found, assuming GPT-2 encodings...
LOVE IS      
Love the dark so his streets in the midnight    
 
I'm got the an time, the peace, I'm coming long for your woman for the smile, my heart and then he'll see it takes, I will be the very past  
I'm dead's a love 
We'll have don't always don't want a little love you 
[Chorus]   
---------------
LOVE IS   
So was a real 
  

   
 
Is a first two  
He been to me 
   
And I'm so like the dark, I'm true's a see down 
 
That was a lot to a hon, oh 
All you can't see   
Let's all the time, but I'm not there
---------------
LOVE IS)  
  
She can't free 
With the girl for your time's a morning, I feel what just for you say, 
If I will what don't go   
Things did I're the day I know I fell here don't want you 
And who'll know I'm on

In [ ]:
!python losstrain.py config/train_lyrics.py \
  --device=cuda \
  --compile=False \
  --dtype=float32 \
  --init_from=resume \
  --out_dir=out-lyrics \
  --max_iters=1000 \
  --log_interval=1 \
  --eval_iters=20 \
  --n_layer=4 \
  --n_head=4 \
  --n_embd=256 \
  --batch_size=4 \
  --block_size=128 \
  --gradient_accumulation_steps=1


Overriding config with config/train_lyrics.py:
# train a miniature character-level model on song lyrics
# good for debugging and playing on macbooks and such

out_dir = 'out-lyrics'
eval_interval = 250  # keep frequent because we'll overfit
eval_iters = 200
log_interval = 10  # don't print too too often

# we expect to overfit on this small dataset, so only save when val improves
always_save_checkpoint = False

wandb_log = False  # override via command line if you like
wandb_project = 'lyrics'
wandb_run_name = 'mini-gpt'

dataset = 'lyrics'
batch_size = 64
block_size = 256  # context of up to 256 previous characters

# baby GPT model :)
n_layer = 6
n_head = 6
n_embd = 384
dropout = 0.2

learning_rate = 1e-3  # with baby networks can afford to go a bit higher
max_iters = 5000
lr_decay_iters = 5000  # make equal to max_iters usually
min_lr = 1e-4  # learning_rate / 10 usually
beta2 = 0.99  # make a bit bigger because number of tokens per iter is small

warmup_iters = 100  # not super nec

In [ ]:
!python sample.py \
  --device=cuda \
  --out_dir=out-lyrics \
  --init_from=resume \
  --start="LOVE IS" \
  --num_samples=3 \
  --max_new_tokens=100


Overriding: device = cuda
Overriding: out_dir = out-lyrics
Overriding: init_from = resume
Overriding: start = LOVE IS
Overriding: num_samples = 3
Overriding: max_new_tokens = 100
number of parameters: 16.03M
No meta.pkl found, assuming GPT-2 encodings...
LOVE IS      
Love the dark so his streets in the midnight    
 
I'm got the an time, the peace, I'm coming long for your woman for the smile, my heart and then he'll see it takes, I will be the very past  
I'm dead's a love 
We'll have don't always don't want a little love you 
[Chorus]   
---------------
LOVE IS   
So was a real 
  

   
 
Is a first two  
He been to me 
   
And I'm so like the dark, I'm true's a see down 
 
That was a lot to a hon, oh 
All you can't see   
Let's all the time, but I'm not there
---------------
LOVE IS)  
  
She can't free 
With the girl for your time's a morning, I feel what just for you say, 
If I will what don't go   
Things did I're the day I know I fell here don't want you 
And who'll know I'm on